<a href="https://colab.research.google.com/github/amyush/FrequencyDomain_Watermarking/blob/main/watermarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setting up the environment**

In [3]:
!git clone https://github.com/amyush/FrequencyDomain_Watermarking.git

Cloning into 'FrequencyDomain_Watermarking'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 24 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [55]:
import numpy as np
import cv2
import math
import pywt
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from PIL import Image as im
import math
from scipy.fftpack import dct
from scipy.fftpack import idct

# **Custom Functions**

In [56]:
def stringToBinary(a):
  l=[]
  res = ''.join(format(ord(i), '08b') for i in a)
  return res

def binaryToString(a):
  return chr(binaryToInt(a))

def integerToBinary(a):
  return int(bin(a)[2:])

def binaryToInt(binary): 
    int_val, i, n = 0, 0, 0
    while(binary != 0): 
        a = binary % 10
        int_val = int_val + a * pow(2, i) 
        binary = binary//10
        i += 1
    return int_val

def calculatePSNR(originalImage, encodedImage):
    originalImage = np.asarray(originalImage)
    encodedImage = np.asarray(encodedImage)
    error = np.sum((originalImage.astype('float') - encodedImage.astype('float')) ** 2)
    error /= float(originalImage.shape[0] * originalImage.shape[1]);
    if error == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(error))

def apply_dct(image_array):
    size = len(image_array[0])
    all_subdct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subpixels = image_array[i:i+8, j:j+8]
            subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")
            all_subdct[i:i+8, j:j+8] = subdct

    return all_subdct

def inverse_dct(all_subdct):
    size = len(all_subdct[0])
    all_subidct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subidct = idct(idct(all_subdct[i:i+8, j:j+8].T, norm="ortho").T, norm="ortho")
            all_subidct[i:i+8, j:j+8] = subidct

    return all_subidct

def process_coefficients(imArray, model, level):
    coeffs=pywt.wavedec2(data = imArray, wavelet = model, level = level)
    coeffs_H=list(coeffs) 
   
    return coeffs_H

# **Steganography**

## **Hiding the data**

In [68]:
def encodingImage(imageName, key, hiddenImageName):
    sizeOrig = 2048
    sizeWatermark = 128
    imgPath ="FrequencyDomain_Watermarking/"
    binKey = stringToBinary(key)
    lenBinKey = len(binKey)
    
    # Reading the cover image
    inputImage = Image.open(imgPath + imageName).convert('L')
    # Resizing the cover image so that it's dimension is compatible with the watermark image
    inputImage = inputImage.resize((sizeOrig,sizeOrig),Image.BILINEAR)
    # Converting the original image to array
    inputArray = np.array(inputImage.getdata(), dtype=np.float).reshape((sizeOrig,sizeOrig))

    # Reading the watermark image
    waterMarkImage = Image.open(imgPath + hiddenImageName).convert('L')
    # Resizing the logo image so that it's dimension is compatible with the cover image
    waterMarkImage = waterMarkImage.resize((sizeWatermark, sizeWatermark), Image.BILINEAR)
    waterMarkArray = np.array(waterMarkImage.getdata(), dtype=np.float).reshape((sizeWatermark,sizeWatermark))

    coeffs_image = process_coefficients(inputArray, 'haar', level=1)

    dctOrigArray = apply_dct(coeffs_image[0])
    dctWaterMarkArray = apply_dct(waterMarkArray)

    flatWaterMark = waterMarkArray.ravel()

    index = -1
    count = 0
    pixel = 0

    for x in range (0, dctOrigArray.shape[0], 16):
        for y in range (0, dctOrigArray.shape[1], 16):
            if count < flatWaterMark.size:
                subdct = dctOrigArray[x:x+16, y:y+16]
                subdct[12][12] = flatWaterMark[count]
                dctOrigArray[x:x+16, y:y+16] = subdct
                count += 1 
    
    coeffs_image[0] = inverse_dct(dctOrigArray)
    image_array_H=pywt.waverec2(coeffs_image, 'haar')
    print(image_array_H)

    resImage = cv2.imwrite('coverRes.png', image_array_H)
    
    # return inputImage, resultArray
    return image_array_H, 'awd'

In [69]:
# coverFileName = input("Enter the cover filename: (eg- Cover_1.png or Cover_2.png)")
# key = input("Enter the key:")
# hiddenImageName = input("Enter the name of the image to be hidden:")

coverFileName = 'cover.jpg'
key = 'Appawdawdawdawdle'
hiddenImageName = 'logo.png'

inputImage, outputImage = encodingImage(coverFileName, key, hiddenImageName)

# plt.subplot(1,2,1),plt.imshow(cv2.cvtColor(ip, cv2.COLOR_BGR2RGB)), plt.title("Input Image")
# plt.subplot(1,2,2),plt.imshow(outputImage), plt.title("Output Image") 
# plt.subplot(1,3,3),plt.imshow(cv2.cvtColor(idct, cv2.COLOR_BGR2RGB)), plt.title("Output Image") 

[[ 4.          3.          3.         ... 11.         12.
  12.        ]
 [ 3.          3.          3.         ... 12.         11.
  12.        ]
 [ 3.          3.          3.         ... 11.         12.
  11.        ]
 ...
 [17.         17.         16.         ... 43.16796875 35.83203125
  33.83203125]
 [14.         13.         12.         ... 35.83203125 42.16796875
  41.16796875]
 [ 7.          6.          6.         ... 34.83203125 40.16796875
  36.16796875]]


In [70]:
re = Image.open('coverRes.png')
ip = Image.open('FrequencyDomain_Watermarking/cover.jpg').convert('L')
ip = ip.resize((2048, 2048), Image.ANTIALIAS)
calculatePSNR(ip, re)

31.740808372870237

# **Results**

In [ ]:
def encode(img_path, wm_path, res_path, alpha):
    img = cv2.imread(img_path)
    img_f = np.fft.fft2(img)
    height, width, channel = np.shape(img)
    watermark = cv2.imread(wm_path)
    wm_height, wm_width = watermark.shape[0], watermark.shape[1]
    x, y = range(height / 2), range(width)
    random.seed(height + width)
    random.shuffle(x)
    random.shuffle(y)
    tmp = np.zeros(img.shape)
    for i in range(height / 2):
        for j in range(width):
            if x[i] < wm_height and y[j] < wm_width:
                tmp[i][j] = watermark[x[i]][y[j]]
                tmp[height - 1 - i][width - 1 - j] = tmp[i][j]
    res_f = img_f + alpha * tmp
    res = np.fft.ifft2(res_f)
    res = np.real(res)
    cv2.imwrite(res_path, res, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

In [ ]:
plt.subplot(1,3,1),plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB)), plt.title("Input Image")
